# Risk Assessment with Unbihexium

[![CI](https://github.com/unbihexium-oss/unbihexium/workflows/CI/badge.svg)](https://github.com/unbihexium-oss/unbihexium/actions)
[![PyPI](https://img.shields.io/pypi/v/unbihexium.svg)](https://pypi.org/project/unbihexium/)
[![License](https://img.shields.io/badge/license-Apache--2.0-blue.svg)](LICENSE.txt)

**Author**: Unbihexium OSS Foundation  
**Version**: 1.0.0

---

## Purpose

Geospatial risk assessment for:
- Flood risk
- Wildfire risk
- Landslide risk
- Seismic hazard

## Risk Framework

$$R = P_{hazard} \times E_{exposure} \times V_{vulnerability}$$

| Factor | Description | Range |
|--------|-------------|-------|
| Hazard | Probability of event | 0-1 |
| Exposure | Assets at risk | 0-1 |
| Vulnerability | Susceptibility | 0-1 |

In [ ]:
import numpy as np
from pathlib import Path

def calculate_risk(hazard: np.ndarray, exposure: np.ndarray, vulnerability: np.ndarray) -> np.ndarray:
    """Calculate composite risk score."""
    return hazard * exposure * vulnerability

## Flood Risk Example

In [ ]:
# Simulate spatial risk factors
np.random.seed(42)
size = (64, 64)

# Hazard: higher near water bodies
hazard = np.zeros(size)
hazard[10:30, 10:50] = 0.8  # River zone
hazard += np.random.rand(*size) * 0.1
hazard = np.clip(hazard, 0, 1)

# Exposure: urban areas
exposure = np.random.rand(*size) * 0.5
exposure[30:50, 20:40] = 0.9  # City center

# Vulnerability: infrastructure age
vulnerability = np.random.rand(*size) * 0.6

# Calculate risk
risk = calculate_risk(hazard, exposure, vulnerability)
print(f"Risk range: [{risk.min():.3f}, {risk.max():.3f}]")
print(f"High risk pixels (>0.3): {(risk > 0.3).sum()}")

## Risk Categories

| Score | Category | Action |
|-------|----------|--------|
| < 0.2 | Low | Monitor |
| 0.2-0.5 | Medium | Prepare |
| > 0.5 | High | Mitigate |

In [ ]:
# Categorize risk
low = (risk < 0.2).sum()
medium = ((risk >= 0.2) & (risk < 0.5)).sum()
high = (risk >= 0.5).sum()

total = risk.size
print(f"Low risk: {low} ({low/total*100:.1f}%)")
print(f"Medium risk: {medium} ({medium/total*100:.1f}%)")
print(f"High risk: {high} ({high/total*100:.1f}%)")

## Using MLP Risk Models

In [ ]:
try:
    import onnxruntime as ort
    
    model_path = Path("../model_zoo/assets/tiny/flood_risk_tiny/model.onnx")
    if model_path.exists():
        session = ort.InferenceSession(str(model_path))
        
        # Prepare input features
        features = np.random.rand(1, 10).astype(np.float32)
        output = session.run(None, {"input": features})
        print(f"Risk prediction: {output[0][0]:.3f}")
    else:
        print("Model not found")
except Exception as e:
    print(f"Error: {e}")

---

**Copyright 2025 Unbihexium OSS Foundation. Apache-2.0 License.**